Do in shell :

`mkdir tmpzip && mkdir tmp && mkdir data && cd data`

`wget -r --no-parent https://data.cquest.org/inpi_rncs/imr/`

`mkdir data.cquest.org/inpi_rncs/imr/flux`

`mv data.cquest.org/inpi_rncs/imr/20* data.cquest.org/inpi_rncs/imr/flux/`

2017:
`for f in *.zip; do mkdir ${f:0:4} && mkdir ${f:0:4}/${f:5:2} && unzip $f -d ${f:0:4}/${f:5:2}/ && rm *.zip && rm *.html; done`

2018 ok

2019, 2020, 2021, 2022
`for f in */*/*.zip; do mkdir ${f:0:2}/${f:3:2}/${f:6:4} && mkdir ${f:0:2}/${f:3:2}/${f:6:4}/${f:11:2} && unzip $f -d ${f:0:2}/${f:3:2}/${f:6:4}/${f:11:2}/; done`
`rm */*/*.html && rm */*/*.zip && rm */*/*.md5`

On ordonne le tout pour que toutes les données soient dans data/TYPE_DONNEES/ANNEE/MOIS/JOUR/GREFFECODE/GREFFECODE2/DONNEESCSV

In [ ]:
import sqlite3
import shutil
import zipfile
import glob
import unidecode
import os
import py7zr
from datetime import date, timedelta
import pandas as pd
import time
from os.path import exists


In [ ]:
sdate = date(2017,5,4)   # start date
edate = date(2022,3,17)   # end date
list_dates = [x.strftime("%Y-%m-%d") for x in pd.date_range(sdate,edate-timedelta(days=1),freq='d').to_list()]

In [ ]:
vol_data = '/Volumes/OnlyData/inpi/data/'

In [ ]:
# Etape de processing des données en amont du lancement de l'enrichissement de la bdd sqlite
def concatFilesRep(type_file, name_concat, pattern):
    pathdate = vol_data + type_file + "/" + ld.split('-')[0] + '/' + ld.split('-')[1] + '/' + ld.split('-')[2] + '/'
    consofile = vol_data + 'synthese/' + ld + name_concat + '.csv'
    # get list files stock
    list_files = glob.glob(pathdate + pattern)
    list_files.sort()
    if len(list_files) > 0:
        
        with open(consofile,"wb") as fout:
            # first file:
            with open(list_files[0], "rb") as f:
                fout.write(f.read())
            # now the rest:    
            for ls in list_files[1:]:
                with open(ls, "rb") as f:
                    try:
                        next(f) # skip the header
                    except:
                        pass
                    fout.write(f.read())
        
        
        #os.system('rm ' + consofile + ' && ulimit -n 10240 && head -n1 ' + list_files[0] + ' > ' + consofile)
        #for ls in list_files:
        #    os.system('tail -n+2 -q ' + ls + ' >> ' + consofile)
        
        print(name_concat + " ok")

In [ ]:
for ld in list_dates[895:]:
    print(ld)
    concatFilesRep("stock", "_stock_rep", "*/*/*_5_rep.csv")
    concatFilesRep("flux-tc", "_flux_rep", "*/*/*_5_rep.csv")
    concatFilesRep("flux-tc", "_flux_rep_nouveau_modifie", "*/*/*6_rep_nouveau_modifie_EVT.csv")
    concatFilesRep("flux-tc", "_flux_rep_partant", "*/*/*7_rep_partant_EVT.csv")
    

In [ ]:
!rm inpi.db

In [ ]:
connection = sqlite3.connect('inpi.db')

In [ ]:
cursor = connection.cursor()

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS rep_pp
              (siren TEXT, rep_noms TEXT, rep_prenoms TEXT, rep_qualite)''')

In [ ]:
cursor.execute('''CREATE TABLE IF NOT EXISTS rep_pm
              (siren TEXT, rep_denomination TEXT, rep_qualite)''')

In [ ]:
cursor.execute('''CREATE INDEX idx_siren_pp ON rep_pp (siren);''')
cursor.execute('''CREATE INDEX idx_siren_pm ON rep_pm (siren);''')
cursor.execute('''CREATE INDEX idx_noms_pp ON rep_pp (rep_noms);''')
cursor.execute('''CREATE INDEX idx_denomination_pm ON rep_pm (rep_denomination);''')

In [ ]:
connection.commit()

In [ ]:
def uniformizeDf(df):
    for c in df.columns:
        df = df.rename(columns={c: unidecode.unidecode(
                                c.lower() \
                                .replace(' ', '') \
                                .replace('.','') \
                                .replace('_','') \
                                .replace('"','') \
                                .replace("'",'')
                    )})

    # Do something
    df['siren'] = df.fillna('')['siren']
    df['rep_noms'] = df.fillna('')['nompatronymique']
    df.loc[df["nomusage"].notna(), "rep_noms"] = df["nompatronymique"] + ' ' + df['nomusage']
    df['rep_noms'] = df['rep_noms'].apply(lambda x: str(x).replace(',','').lower())
    df['rep_prenoms'] = df.fillna('')['prenoms']
    df['rep_prenoms'] = df['rep_prenoms'].apply(lambda x: str(x).replace(',','').lower())
    df['rep_denomination'] = df.fillna('')['denomination']
    df['rep_qualite'] = df.fillna('')['qualite']
    dfpp = df[df['type'].str.contains('Physique')][['siren','rep_noms','rep_prenoms','rep_qualite']]
    dfpm = df[df['type'].str.contains('Morale')][['siren','rep_denomination','rep_qualite']]

    return dfpp, dfpm


In [ ]:
# On s'occupe du stock initial (car vraiment gros et pas adapté à la méthode ci-dessous)
list_files = glob.glob(vol_data + 'stock/2017/05/04/*/*/*5_rep.csv')
list_files.sort()
for lf in list_files:
    print(lf)
    df = pd.read_csv(lf,sep=";",dtype=str)
    dfpp, dfpm = uniformizeDf(df)
    dfpp.set_index('siren').to_sql('rep_pp', con=connection, if_exists='append')
    dfpm.set_index('siren').to_sql('rep_pm', con=connection, if_exists='append')

In [ ]:
original = 'inpi.db'
target = 'inpi-stock-initial.db'
shutil.copyfile(original, target)

In [ ]:
list_dates[698]

In [ ]:
len(list_dates)

In [ ]:
# Toutes les dates après la date du stock initial
for ld in list_dates[1:]:
    print(ld)
    if ld.split('-')[2] == '01':
        original = 'inpi.db'
        target = 'inpi-'+ld+'.db'
        shutil.copyfile(original, target)
    
    consofile = vol_data + 'synthese/' + ld + '_stock_rep.csv'
    # Manage stocks
    if(os.path.exists(consofile)):
        df = pd.read_csv(consofile,sep=";",dtype=str,on_bad_lines='warn')
        dfpp, dfpm = uniformizeDf(df)
        print('loaded and uniformize')
        # Delete old stock
        if(ld != '2017-05-04'):
            print('Old stock')
            result = 0
            for index,row in dfpp.iterrows():
                del_query = """DELETE from rep_pp where siren = ?"""
                cursor.execute(del_query, (row['siren'],))
                result = result + cursor.rowcount
            connection.commit()
            for index,row in dfpp.iterrows():
                del_query = """DELETE from rep_pm where siren = ?"""
                cursor.execute(del_query, (row['siren'],))
                result = result + cursor.rowcount
            connection.commit()
            print('Old Stock deleted : ' + str(result) + ' deleted records')
        # Add new stock
        dfpp.set_index('siren').to_sql('rep_pp', con=connection, if_exists='append')
        dfpm.set_index('siren').to_sql('rep_pm', con=connection, if_exists='append')
        print('Stock processed : ' + str(dfpp.shape[0]+dfpm.shape[0]) + 'added records')

    
    consofile = vol_data + 'synthese/' + ld + '_flux_rep.csv'
    if(os.path.exists(consofile)):
        df = pd.read_csv(consofile,sep=";",dtype=str,on_bad_lines='warn')
        dfpp, dfpm = uniformizeDf(df)
        dfpp.set_index('siren').to_sql('rep_pp', con=connection, if_exists='append')
        dfpm.set_index('siren').to_sql('rep_pm', con=connection, if_exists='append')
        print('Flux rep processed : ' + str(dfpp.shape[0]+dfpm.shape[0]) + 'added records')

    consofile = vol_data + 'synthese/' + ld + '_flux_rep_nouveau_modifie.csv'
    if(os.path.exists(consofile)):
        df = pd.read_csv(consofile,sep=";",dtype=str,on_bad_lines='warn')
        dfpp, dfpm = uniformizeDf(df)
        dfpp.set_index('siren').to_sql('rep_pp', con=connection, if_exists='append')
        dfpm.set_index('siren').to_sql('rep_pm', con=connection, if_exists='append')
        print('Flux rep modified new processed : ' + str(dfpp.shape[0]+dfpm.shape[0]) + 'added records')

    
    consofile = vol_data + 'synthese/' + ld + '_flux_rep_partant.csv'
    if(os.path.exists(consofile)):
        df = pd.read_csv(consofile,sep=";",dtype=str,on_bad_lines='warn')
        dfpp, dfpm = uniformizeDf(df)
        # Delete each rep
        result = 0
        for index,row in dfpp.iterrows():
            del_query = """DELETE from rep_pp where siren = ? AND rep_noms = ? AND rep_prenoms = ? AND rep_qualite = ?"""
            cursor.execute(del_query, (row['siren'], row['rep_noms'], row['rep_prenoms'], row['rep_qualite']))
            result = result + cursor.rowcount
        connection.commit()
        for index,row in dfpm.iterrows():
            del_query = """DELETE from rep_pm where siren = ? AND rep_denomination = ? AND rep_qualite = ?"""
            cursor.execute(del_query, (row['siren'], row['rep_denomination'], row['rep_qualite']))
            result = result + cursor.rowcount
        connection.commit()
        print('Flux rep partant processed : ' + str(result) + ' deleted records')

        


In [ ]:
conn = sqlite3.connect('inpi.db', isolation_level=None,
                       detect_types=sqlite3.PARSE_COLNAMES)
print('connected')

db = pd.read_sql_query("SELECT * FROM rep_pm", conn)
print('read rep_pm ok')
db = db.drop_duplicates(keep="first")
print('deduplicates ok')
dbpm = db.groupby(['siren','rep_denomination'],as_index=False)['rep_qualite'].agg(lambda col: ', '.join(col))
print('groupby ok')
dbpm[dbpm['siren'] != ''].to_csv('rep_pm.csv',index=False)
print('csv saved ok')

db = pd.read_sql_query("SELECT * FROM rep_pp", conn)
print('read rep_pp ok')
db = db.drop_duplicates(keep="first")
print('deduplicates ok')
dbpp = db.groupby(['siren','rep_noms','rep_prenoms'],as_index=False)['rep_qualite'].agg(lambda col: ', '.join(col))
print('groupby ok')
dbpp[dbpp['siren'] != ''].to_csv('rep_pp.csv',index=False)
print('csv saved ok')
